In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import cv2
from google.colab import files

In [ ]:
#Load Dataset
(train_images,train_labels), (test_images, test_labels) = mnist.load_data()

In [ ]:
print(train_images.shape)

(60000, 28, 28)


In [ ]:
#Normalize images to be in between 0 & 1 and reshape them
train_images = train_images.astype('float32')/255.0
test_images = test_images.astype('float32')/255.0
train_images = train_images.reshape(train_images.shape[0],28,28,1)
test_images = test_images.reshape(test_images.shape[0],28,28,1)

In [ ]:
print(train_images.shape)
#1 > gray scale > 1 channel
# convolution requires the number of channels hence we reshape
# for an RGB image the channels are 3 hence channel will be 3

(60000, 28, 28, 1)


In [ ]:
#Convert labels to one hot encoded format
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)
#to_categorical > converts in to integer labels

In [ ]:
#Buliding the CNN model
#Instantiating the Sequential model

model= Sequential()

#Add layers one by one
#Parameters >   Weights > learnable
#Hyperparameters are not learnable > its defined based on experiments
#(3,3) filter - learnable
model.add(Conv2D(32,(3,3),activation = 'relu',input_shape=(28,28,1))) #input shape for 1 image #Convolution Layer
model.add(MaxPooling2D((2,2))) #Maxpooling layer filters/parameters > (2,2) kernel size #Not learnable > max in the pool
model.add(Conv2D(64,(3,3),activation='relu'))#conv layer with 64 fiters of (3,3) size
model.add(MaxPooling2D((2,2)))
model.add(Flatten()) # Flatten > to convert to 1D; done before feeding in to fully connected layer
model.add(Dense(128,activation='relu'))#no of neurons 128
model.add(Dropout(0.5)) #Half of the neurons get dropped randomly as 0.5 dropout > dropout is done to avoid overfitting and make the model simpler
#There's only one hidden layer in this network
model.add(Dense(10,activation='softmax'))#op layer > softmax is usually used in op layer if the op is non-binary
#If op is binary we use Sigmoid

In regression there is no activation function > we use activation function in case of non-continuous cases

In [ ]:
#Compiling the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
#We are using adam optimizer > adaptive, loss function, we use metrics as accuracy (Whatever predicted correctly divided by total)

In [ ]:
#Train the model #epoch > 1 iteration through entire dataset
model.fit(train_images, train_labels, epochs=5, batch_size=32)


Epoch 1/5
1875/1875 [==============================] - 59s 31ms/step - loss: 0.1946 - accuracy: 0.9412
Epoch 2/5
1875/1875 [==============================] - 56s 30ms/step - loss: 0.0753 - accuracy: 0.9778
Epoch 3/5
1875/1875 [==============================] - 56s 30ms/step - loss: 0.0568 - accuracy: 0.9837
Epoch 4/5
1875/1875 [==============================] - 55s 30ms/step - loss: 0.0445 - accuracy: 0.9862
Epoch 5/5
1875/1875 [==============================] - 57s 30ms/step - loss: 0.0382 - accuracy: 0.9887


In [ ]:
#Save the trained model
model.save('mnist_cnn_model.keras')

In [ ]:
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_accuracy}')
print(f'Test loss: {test_loss}')

313/313 [==============================] - 3s 9ms/step - loss: 0.0265 - accuracy: 0.9918
Test accuracy: 0.9918000102043152
Test loss: 0.02648916095495224
